In [27]:
import jupyter_aws_sso

import importlib
importlib.reload(jupyter_aws_sso)

jupyter_aws_sso.login("Jupyter-IR-AdministratorAccess", "383086473915")

Profile found
If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=FMGQ-KXBS to activate the session


<IPython.core.display.Javascript object>

## Parameters
The following cell holds the parameters for the environment

* **athena_database**: The name of the athena database that will be used to query logs in S3.
* **athena_cloudtrail_table**: The name of the athena table that will be used to query CloutTrail logs in S3.
* **flow_logs_bucket**: The bucket that holds the organizational flow logs. Leave empty and one will be created automatically.
* **enable_flow_logs_automatically**: Automatically enable flow logs in all VPCs in the organization. See [blog](https://aws.amazon.com/blogs/mt/how-to-enable-vpc-flow-logs-automatically-using-aws-config-rules/)


In [28]:
# Save Settings
athena_database = "jupyter-incident-response"  # <--- To customize, replace the value with the athena database name
athena_cloudtrail_table = "cloudtrail_logs"  # <--- To customize, replace the value with the athena table name for cloudtrail logs
flow_logs_bucket = ""  # <--- Leave blank and a bucket will be created automatically
enable_flow_logs_automatically = True


# Organizational Onboarding

The notebook will validate and update the organization to configure the logging and configuration of the organization to the other runbooks have access to the data and services needed to execute the runbooks.

This runbook is designed to be able to be run multiple times, so it will not create additional resources if the configurate has already been found.

The following services will be configured.
* **CloudTrail**: An organization trail will be configured to store the CloudTrail logs in a bucket in the management account. The first trail in each account is included free of charge. If an organzation trail already exists, a new one will not be created.
* **VPC Flow Logs**: Any VPCs with the tag 'runbooks-enable-flowlogs' set to 'true' will have VPC flow logs enabled into a central bucket.
* **Athena**: Athena will be configured to query the following service logs:
  * CloudTrail
  * VPC Flow Logs
  * ***Future*** Application Load Balancer
  * ***Future*** Elastic Load Balancer
  * ***Future*** CloudFront
  * ***Future*** Amazon EMR
  * ***Future*** Global Accelerator
  * ***Future*** GuardDuty
  * ***Future*** Network Firewall
  * ***Future*** Network Load Balancer
  * ***Future*** Route 53
  * ***Future*** Amazon SES
  * ***Future*** AWS WAF
  

## Load Settings

The information on how to connect to data in the AWS environment is managed through SSM Parameter Store.

The athena database and tablenames can be provided below. The runbook will manage the Athena tables so they are configured with the columns and partitioning expected by the notebooks.

In [29]:
# Load Settings
import boto3
ssm_client = boto3.client('ssm')

put_response = ssm_client.put_parameter(Name='Jupyter-Athena-Database', Value=athena_database, Type="String", Overwrite=True)
sso_portal_url_response = ssm_client.put_parameter(Name='Jupyter-Athena-Cloudtrail', Value=athena_cloudtrail_table, Type="String", Overwrite=True)

## Validate CloudTrail
The next section will check the CloudTrail logs and validate an organizational trail is configured.

In [30]:


found_org_trail = False
org_trail_info = {}

cloudtrail_client = boto3.client('cloudtrail', region_name = "us-east-1")

paginator = cloudtrail_client.get_paginator('list_trails')

response_iterator = paginator.paginate()
for page in response_iterator:
    for trail in page["Trails"]:
        trail_response = cloudtrail_client.get_trail(Name=trail['TrailARN'])
        if trail_response['Trail']['IsOrganizationTrail'] and trail_response['Trail']['IsMultiRegionTrail']:
            found_org_trail = True
            org_trail_info = trail_response['Trail']

            
if found_org_trail:
    print(f"Organization Trail was found: {org_trail_info['Name']} in region {org_trail_info['HomeRegion']}")
else:
    print("Organization Trail was NOT found, continue to create one. Otherwise, check your existing configuration and re-run this cell.")

Organization Trail was found: trail-organization-trail in region us-east-1


### Create Trail, if needed
If an organization trail was not found, the following section will create one using a cloudformation template.

In [31]:
cfn_client = boto3.client('cloudformation')

if not found_org_trail:
    print("Create an organizational trail via CloudFormation")
    !aws cloudformation deploy --template-file cfn-templates/organization-trail.yaml --stack-name organization-trail
    trail_arn = !aws cloudformation describe-stacks --stack-name organization-trail --query "Stacks[0].Outputs[?OutputKey=='TrailArn'].OutputValue" --output text
    trail_response = cloudtrail_client.get_trail(Name=trail_arn[0])
    org_trail_info = trail_response['Trail']
    
print(f"org_trail_info = {org_trail_info['Name']}")
print(f"S3 Bucket: {org_trail_info['S3BucketName']}")

org_client = boto3.client('organizations')

desc_client = org_client.describe_organization()

if not "S3KeyPrefix" in org_trail_info:
    org_trail_info['S3KeyPrefix'] = ''

org_trail_info['s3_url'] = f"s3://{org_trail_info['S3BucketName']}{org_trail_info['S3KeyPrefix']}/AWSLogs/{desc_client['Organization']['Id']}"    

org_trail_info['s3_url']

s3_url = org_trail_info['s3_url']
s3_bucket = org_trail_info['S3BucketName']
s3_prefix = org_trail_info['S3KeyPrefix']
s3_bucket = org_trail_info['S3BucketName']


org_trail_info = trail-organization-trail
S3 Bucket: organization-trail-trailbucket-1nou196wsxixy


## Optional - Configure auto-remediation of Flow Logs

* Create central bucket
* Cloud Formation to set up stack set with auto-remediation of VPC flow logs to central 
* bucket/AWSLogs/vpcflowlogs/region/year/month/day

# Create / Update the Athena Table for CloudTrail
The next section will create a CloudFormation stack that configures the Athena cloudtrail tables.


In [32]:

!aws cloudformation deploy --stack-name cloudtrail-athena --template-file cfn-templates/athena-cloudtrail.yaml --parameter-overrides TrailBucket=$s3_bucket S3Prefix=$s3_prefix DatabaseName=$athena_database TableName=$athena_cloudtrail_table



Waiting for changeset to be created..

No changes to deploy. Stack cloudtrail-athena is up to date


## Test CloudTrail access with Athena
The following code uses a helper library to query cloudtrail logs and return them as a dataframe

In [33]:
import jupyter_athena
import importlib
importlib.reload(jupyter_aws_sso)

accountid = "383086473915"

sql = f"""
SELECT * 
FROM \"{athena_database}\".\"{athena_cloudtrail_table}\"
WHERE accountId = '{accountid}'
LIMIT 10
"""

df = jupyter_athena.run_query(sql)
df

OperationalError: Permission denied on S3 path: s3://organization-trail-trailbucket-1nou196wsxixy/AWSLogs/o-krcf45heml/383086473915/CloudTrail/us-east-1/2022/10/15/383086473915_CloudTrail_us-east-1_20221015T1355Z_iAHECKf61BnT1pgy.json.gz

## Flow Logs
Enable athena to query organizational flow logs.

This needs to be deployed in two steps. 
1. The central bucket and athena query
1. The stacksets for all regions/accounts in organization.

In [ ]:
flow_logs_bucket=""

In [ ]:

!aws cloudformation deploy --stack-name flowlogs-athena --template-file cfn-templates/organization-flow-logs.yaml --parameter-overrides DatabaseName=$athena_database FlowLogsBucket=$flow_logs_bucket --capabilities CAPABILITY_IAM

    

In [40]:
stack_set_name = "jupyter-incident-response"
stack_set_info = {}

cloudformation_client = boto3.client('cloudformation')
try:
    describe_stack_set_response = cloudformation_client.describe_stack_set(
        StackSetName=stack_set_name,
        CallAs='SELF'
    )
    stack_set_info= describe_stack_set_response['StackSet']
except cloudformation_client.exceptions.StackSetNotFoundException:
    print('stackset not found, create it')

    response = cloudformation_client.create_stack_set(
        StackSetName=stack_set_name,
        Description='The stackset to configure accounts and regions for incident response runbooks.',
        TemplateBody='string',
        TemplateURL='string',
        Parameters=[
            {
                'ParameterKey': 'string',
                'ParameterValue': 'string',
                'UsePreviousValue': True|False,
                'ResolvedValue': 'string'
            },
        ],
        Capabilities=[
            'CAPABILITY_IAM',
        ],
        Tags=[
            {
                'Key': 'string',
                'Value': 'string'
            },
        ],
        PermissionModel='SERVICE_MANAGED',
        AutoDeployment={
            'Enabled': True,
            'RetainStacksOnAccountRemoval': False
        },
        CallAs='SELF',
        ManagedExecution={
            'Active': True
        }
    )
    
    describe_stack_set_response = cloudformation_client.describe_stack_set(
        StackSetName=stack_set_name,
        CallAs='SELF'
    )
    stack_set_info= describe_stack_set_response['StackSet']

stackset not found, create it


ClientError: An error occurred (ValidationError) when calling the CreateStackSet operation: Only one of TemplateUrl, TemplateBody and StackSetName must be specified.